##### *Copyright 2020 Google LLC*
*Licensed under the Apache License, Version 2.0 (the "License")*

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Retrain a detection model for Edge TPU with quant-aware training (TF 1.12)

This notebook uses a set of TensorFlow training scripts to perform transfer-learning on a quantization-aware object detection model and then convert it for compatibility with the [Edge TPU](https://coral.ai/products/).

Specifically, this tutorial shows you how to retrain a MobileNet V1 SSD model so that it detects two pets: Abyssinian cats and American Bulldogs (from the [Oxford-IIIT Pets Dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/)), using TensorFlow r1.12.

Beware that, compared to a desktop computer, this training can take *a lot* longer in Colab because Colab provides limited resources for long-running operations. So you'll likely see faster training speeds if you [connect this notebook to a local runtime](https://research.google.com/colaboratory/local-runtimes.html), or instead follow the [tutorial to run this training in Docker](https://coral.ai/docs/edgetpu/retrain-detection/) (which includes more documentation about this process).

## Import TensorFlow

In [ ]:
! pip3 uninstall tensorflow -y
#! pip3 install tensorflow==1.12
! pip3 install tensorflow==1.15

In [ ]:
import tensorflow as tf
print(tf.__version__)

## Clone the model and training repos

In [ ]:
! git clone https://github.com/tensorflow/models.git

In [ ]:
! cd models && git checkout f788046ca876a8820e05b0b48c1fc2e16b0955bc

In [ ]:
! git clone https://github.com/google-coral/tutorials.git

! cp -r tutorials/docker/object_detection/scripts/* models/research/

## Import dependencies

For details, see https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md

In [ ]:
! apt-get install -y python python-tk
! pip install Cython contextlib2 pillow lxml jupyter matplotlib

In [ ]:
# Get protoc 3.0.0, rather than the old version already in the container
! wget https://www.github.com/google/protobuf/releases/download/v3.0.0/protoc-3.0.0-linux-x86_64.zip
! unzip protoc-3.0.0-linux-x86_64.zip -d proto3
! mkdir -p local/bin && mkdir -p local/include
! mv proto3/bin/* local/bin
! mv proto3/include/* local/include
! rm -rf proto3 protoc-3.0.0-linux-x86_64.zip

In [ ]:
# Install pycocoapi
! git clone --depth 1 https://github.com/cocodataset/cocoapi.git
! (cd cocoapi/PythonAPI && make -j8)
! cp -r cocoapi/PythonAPI/pycocotools/ /content/models/research/
! rm -rf cocoapi

In [ ]:
# Run protoc on the object detection repo (generate .py files from .proto)
% cd /content/models/research/
#! ../../local/bin/protoc object_detection/protos/*.proto --python_out=.
! /content/local/bin/protoc object_detection/protos/*.proto --python_out=.

In [ ]:
import os
os.environ['PYTHONPATH'] += ":/content/models/research:/content/models/research/slim"

Just to verify everything is correctly set up:

In [ ]:
! python object_detection/builders/model_builder_test.py

## Convert training data to TFRecord

In [ ]:

# Import Dataset from Github
repo_url = 'https://github.com/MBavelock/object_detection_demo'

import os

%cd /content

repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))

!git clone {repo_url}
%cd {repo_dir_path}
!git pull

#Upload Dataset as zip
from google.colab import drive
drive.mount('/content/gdrive')

import zipfile
zip_ref = zipfile.ZipFile("/content/gdrive/MyDrive/Dataset/images.zip", 'r')
zip_ref.extractall("/content/object_detection_demo/data")
zip_ref.close()

#!unzip '/content/gdrive/MyDrive/Critter_Scatterer_-_Senior Design/ECE_458/Dataset/images.zip' -d '/content/object_detection_demo/data'


# Convert train folder annotation xml files to a single csv file,
# generate the `label_map.pbtxt` file to `data/` directory as well.
!python xml_to_csv.py -i data/images/train -o data/annotations/train_labels.csv -l data/annotations

# Convert test folder annotation xml files to a single csv.
!python xml_to_csv.py -i data/images/test -o data/annotations/test_labels.csv

# Generate `train.record`
!python generate_tfrecord.py --csv_input=data/annotations/train_labels.csv --output_path=data/annotations/train.record --img_path=data/images/train --label_map data/annotations/label_map.pbtxt

# Generate `test.record`
!python generate_tfrecord.py --csv_input=data/annotations/test_labels.csv --output_path=data/annotations/test.record --img_path=data/images/test --label_map data/annotations/label_map.pbtxt


test_record_fname = '/content/object_detection_demo/data/annotations/test.record'
train_record_fname = '/content/object_detection_demo/data/annotations/train.record'
label_map_pbtxt_fname = '/content/object_detection_demo/data/annotations/label_map.pbtxt'

To train with different images, read [how to configure your own training data](https://coral.ai/docs/edgetpu/retrain-detection/#configure-your-own-training-data).

In [ ]:
# in research
#OBJ_DET_DIR="$PWD"
#LEARN_DIR="${OBJ_DET_DIR}/learn_pet"
#DATASET_DIR="${LEARN_DIR}/pet"
#CKPT_DIR="${LEARN_DIR}/ckpt"
#TRAIN_DIR="${LEARN_DIR}/train"
#OUTPUT_DIR="${LEARN_DIR}/models"

#! ./prepare_checkpoint_and_dataset.sh --network_type mobilenet_v1_ssd --train_whole_model false

### NEED TODO 
#cp train.record -> /content/models/research/learn_pet/pet/
#cp test.record -> /content/models/research/learn_pet/pet/
#cp label_map.pbtxt -> /content/models/research/learn_pet/pet/
#change configfile to be 12 labels
#change configfile to match train.record, test.record, label_map



# Download checkpoint for transfer learning
! echo 'PREPARING checkpoint...'
%mkdir -p /content/models/research/learn_pet
%cd /content/models/research/learn_pet
! wget -O ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18.tar.gz http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18.tar.gz
! tar zxvf ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18.tar.gz
%rm /content/models/research/learn_pet/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18.tar.gz

# Add Config file to dir
#! echo 'CHOSING config file...'
#%cd /content/models/research
#%mkdir -p /content/models/research/learn_pet/ckpt/
#%cp configs/pipeline_mobilenet_v1_ssd_retrain_last_few_layers.config /content/models/research/learn_pet/ckpt/pipeline.config

# Edit Config File
import re
#pipeline_fname = '/content/models/research/learn_pet/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18/pipeline.config'
pipeline_fname = '/content/object_detection_demo/pipeline.config'
train_record_fname = train_record_fname
test_record_fname = test_record_fname
label_map_pbtxt_fname = label_map_pbtxt_fname
fine_tune_checkpoint = os.path.join('/content/models/research/learn_pet/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18/', "model.ckpt")
##### Copy old check point if you want to keep training
#%mkdir "/content/models/research/learn_pet/finetuning"
#%cp -av "/content/gdrive/MyDrive/TFLite/trainingckpt/" "/content/models/research/learn_pet/finetuning/"
#%mv "/content/models/research/learn_pet/model.ckpt-50000.data-00000-of-00001" "/content/models/research/learn_pet/model.ckpt"
#fine_tune_checkpoint = "/content/models/research/learn_pet/finetuning/trainingckpt/model.ckpt-50000"

batch_size = 12
num_steps = 50000
num_classes = 12

with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    # Set max_negatives_per_positive: maximum number of negatives 
    # to retain for each positive anchor. By default, num_negatives_per_positive is None, which 
    # means that we do not enforce a prespecified negative:positive ratio. Note also that 
    # num_negatives_per_positives can be a float (and will be converted to be a float even if it is 
    # passed in otherwise).
    s = re.sub('max_negatives_per_positive: [0-9]+',
               'max_negatives_per_positive: {}'.format(3), s)

    # Set min_negatives_per_image: minimum number of negative anchors to 
    # sample for a given image. Setting this to a positive number allows sampling negatives in an image 
    # without any positive anchors and thus not biased towards at least one detection per image.
    s = re.sub('min_negatives_per_image: [0-9]+',
               'min_negatives_per_image: {}'.format(1), s)

    f.write(s)


!cat {pipeline_fname}

#%cp "/content/gdrive/MyDrive/TFLite/test/test.record" "/content/models/research/learn_pet/"
#%cp "/content/gdrive/MyDrive/TFLite/train/train.record" "/content/models/research/learn_pet/"
#%cp "/content/gdrive/MyDrive/TFLite/train/label_map.pbtxt" "/content/models/research/learn_pet/"

#! echo 'PREPARING dataset'
#%mkdir -p /content/models/research/learn_pet/pet
#%cd /content/models/research/learn_pet/pet

#! echo 'PREPARING label map...'
#%cd /content/models/research
#%cp object_detection/data/pet_label_map.pbtxt /content/models/research/learn_pet/pet


## Perform transfer-learning

The following script takes several hours to finish in Colab. (You can shorten by reducing the steps, but that reduces the final accuracy.)

If you didn't already select "Run all" then you should run all remaining cells now. That will ensure the rest of the notebook completes while you are away, avoiding the chance that the Colab runtime times-out and you lose the training data before you download the model.

In [ ]:
#%env NUM_TRAINING_STEPS=500
#NUM_TRAINING_STEPS=num_steps
#%env NUM_EVAL_STEPS=100
#NUM_EVAL_STEPS=num_steps/100

# If you're retraining the whole model, we suggest thes values:
# %env NUM_TRAINING_STEPS=50000
# %env NUM_EVAL_STEPS=2000


%mkdir -p /content/models/research/learn_pet/train


In [ ]:
# https://github.com/google-coral/tutorials/tree/master/docker/object_detection/scripts
#! ./retrain_detection_model.sh --num_training_steps $NUM_TRAINING_STEPS --num_eval_steps $NUM_EVAL_STEPS

! python /content/models/research/object_detection/model_main.py \
  --pipeline_config_path="/content/object_detection_demo/pipeline.config" \
  --model_dir="/content/models/research/learn_pet/train" \
  --num_train_steps=100000 \
  --num_eval_steps=2000

  # 500
  # 100
  
# If not using custome pipefile.config
# "/content/models/research/learn_pet/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18/pipeline.config"


In [ ]:
# Download
!cp -av /content/models/research/learn_pet/train /content/gdrive/MyDrive/TFLite/trainingckpt_413
!cp /content/models/research/learn_pet/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18/pipeline.config /content/gdrive/MyDrive/TFLite/trainingckpt

As training progresses, you can see new checkpoint files appear in the `models/research/learn_pet/train/` directory.

## Compile for the Edge TPU

In [ ]:
#! ./convert_checkpoint_to_edgetpu_tflite.sh --checkpoint_num $NUM_TRAINING_STEPS


#echo "GENERATING label file..."
#echo "0 Abyssinian" >> "${OUTPUT_DIR}/labels.txt"
#echo "1 american_bulldog" >> "${OUTPUT_DIR}/labels.txt"

!echo "EXPORTING frozen graph from checkpoint..."
!python /content/models/research/object_detection/export_tflite_ssd_graph.py \
  --pipeline_config_path="/content/object_detection_demo/pipeline.config" \
  --trained_checkpoint_prefix="/content/models/research/learn_pet/train/model.ckpt-100000" \
  --output_directory="/content/" \
  --add_postprocessing_op=true

!echo "CONVERTING frozen graph to TF Lite file..."
!tflite_convert \
  --output_file="/content/output_tflite_graph.tflite" \
  --graph_def_file="/content/tflite_graph.pb" \
  --inference_type=QUANTIZED_UINT8 \
  --input_arrays="normalized_input_image_tensor" \
  --output_arrays="TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3" \
  --mean_values=128 \
  --std_dev_values=128 \
  --input_shapes=1,300,300,3 \
  --change_concat_input_ranges=false \
  --allow_nudging_weights_to_use_fast_gemm_kernel=true \
  --allow_custom_ops

!echo "TFLite graph generated at /content/output_tflite_graph.tflite"


In [ ]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

! sudo apt-get update

! sudo apt-get install edgetpu-compiler	

In [ ]:
%cd /content/

! ls

In [ ]:
%cd /content/
!edgetpu_compiler -s output_tflite_graph.tflite

# Download
!cp /content/output_tflite_graph.tflite /content/gdrive/MyDrive/TFLite/

Download the files:

In [ ]:
from google.colab import files

files.download('output_tflite_graph_edgetpu.tflite')
#files.download('labels.txt')

If you get a "Failed to fetch" error here, it's probably because the files weren't done saving. So just wait a moment and try again.

Also look out for a browser popup that might need approval to download the files.

## Run the model on the Edge TPU




You can now run the model on your Coral device with acceleration on the Edge TPU.

To get started, try using [this code for object detection with the TensorFlow Lite API](https://github.com/google-coral/tflite/tree/master/python/examples/detection). Just follow the instructions on that page to set up your device, copy the `output_tflite_graph_edgetpu.tflite` and `labels.txt` files to your Coral Dev Board or device with a Coral Accelerator, and pass it a photo to see the detected objects.

Check out more examples for running inference at [coral.ai/examples](https://coral.ai/examples/#code-examples/).

## Implementation details



All the scripts used in this notebook come from the following locations:<br>
+  https://github.com/google-coral/tutorials/tree/master/docker/object_detection/scripts
+  https://github.com/tensorflow/models/tree/r1.13.0/research/object_detection/

More explanation of the steps in this tutorial is available at
https://coral.ai/docs/edgetpu/retrain-detection/.